# 16S Model Training and Evaluation

**Objective:** To build, train, and evaluate a deep learning classifier for the 16S rRNA gene using the pre-processed data.

**Methodology:**
1. Load the training/testing data and encoders from disk.
2. Define the neural network architecture using TensorFlow/Keras.
3. Train the model on the training data, using the GPU if available.
4. Evaluate the final model's accuracy on the unseen test data.

In [1]:
import numpy as np
import tensorflow as tf
from scipy.sparse import load_npz
import pickle
from pathlib import Path
import sys

# Set up project path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# --- Verification Step: Check for GPU ---
# This will tell us if TensorFlow can see your GPU.
print("--- TensorFlow Setup ---")
print(f"TensorFlow Version: {tf.__version__}")
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"✅ GPU detected: {gpu_devices[0]}")
else:
    print("⚠️ No GPU detected. TensorFlow will run on CPU.")
print("-" * 26)

--- TensorFlow Setup ---
TensorFlow Version: 2.10.1
✅ GPU detected: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
--------------------------
